In [1]:
#!pip install datefinder

In [2]:
import pandas as pd
import numpy as np
import datefinder as dfi
import datetime
from pandas import ExcelWriter

In [3]:
################################################
#FUNCIONES NO IMPORTADAS: PASAR A .PY E IMPORTAR
################################################

def del_p(x):
    ''' This function which takes a string as argument, returns the same string
    that was entered but without (p) in case there are any of these in the original
    string
    '''
    try:
        return x.replace('(p)', '')
    except:
        pass
    
def del_total(x):
    ''' This function which takes a string as argument, returns the same string
    that was entered but without (p) in case there are any of these in the original
    string
    '''
    try:
        return x.replace('_Total', '')
    except:
        pass
    
def del_acumulado(x):
    ''' This function which takes a string as argument, returns the same string
    that was entered but without (p) in case there are any of these in the original
    string
    '''
    try:
        return x.replace('_Acumulado', '')
    except:
        pass

def del_years(x):
    ''' This function returns a nan if the given argument (must be a string)
    has 4 or less characters. Otherwise  it returns the intact original string.
    The original purpose of this function was to delete cells with year info but 
    no month info.
    '''
    import numpy as np
    if len(x.strip()) < 5:
        return 'tbd'
    else:
        return x
    
def type_el(x):
    '''Returns the type of an element'''
    print(type(x))
    
def del_spaces(x):
    ''' Function to apply the strip method for strings'''
    return x.strip()

def translate_month(x):
    if 'Enero' in x:
        return x.replace('Enero', 'January')
    elif 'Febrero' in x:
        return x.replace('Febrero', 'February')
    elif 'Marzo' in x:
        return x.replace('Marzo', 'March')
    elif 'Abril' in x:
        return x.replace('Abril', 'April')
    elif 'Mayo' in x:
        return x.replace('Mayo', 'May')
    elif 'Junio' in x:
        return x.replace('Junio', 'June')
    elif 'Julio' in x:
        return x.replace('Julio', 'July')
    elif 'Agosto' in x:
        return x.replace('Agosto', 'August')
    elif 'Septiembre' in x:
        return x.replace('Septiembre', 'September')
    elif 'Octubre' in x:
        return x.replace('Octubre', 'October')
    elif 'Noviembre' in x:
        return x.replace('Noviembre', 'November')
    elif 'Diciembre' in x:
        return x.replace('Diciembre', 'December')
    else:
        return x

def translate_number(x):
    if 'M01' in x:
        return x.replace('M01', ' January')
    elif 'M02' in x:
        return x.replace('M02', ' February')
    elif 'M03' in x:
        return x.replace('M03', ' March')
    elif 'M04' in x:
        return x.replace('M04', ' April')
    elif 'M05' in x:
        return x.replace('M05', ' May')
    elif 'M06' in x:
        return x.replace('M06', ' June')
    elif 'M07' in x:
        return x.replace('M07', ' July')
    elif 'M08' in x:
        return x.replace('M08', ' August')
    elif 'M09' in x:
        return x.replace('M09', ' September')
    elif 'M10' in x:
        return x.replace('M10', ' October')
    elif 'M11' in x:
        return x.replace('M11', ' November')
    elif 'M12' in x:
        return x.replace('M12', ' December')
    else:
        return x

    
def datematcher(x):
    '''Function to extract a datetime object from a string
    (Based on the datefinder library)'''
    matches = dfi.find_dates(x)
    for match in matches:
        return(match)
    
    
def dateconverter(x):  
    ''' Converts a datetime.datetime object into a  datetime.date object and returns it'''
    return datetime.datetime.date(x)

def index_b_count(w, x, v, h):
    '''
    Return a dictionary containing the ratio between the count of two given categorical variables for the rows of a df
    whose index has a given value as value, and the index of refference of the dataframe as key
    Where w is a dataframe, x is a list of indices, v is the column that will serve as basis for the count
    and h is the column from which the values making up x have been taken
    '''
    emptydict = {}
    
    
    for i in x:
        w_temp = w[w[h] == i]
        if len(w_temp[v].value_counts().value_counts()) > 1:
            a = len(w_temp[v])
            b = w_temp[v].value_counts()[1]
            c = b/a
            emptydict[i] = c
        else:
            emptydict[i] = 0
            
    
    return emptydict

def punct_num(x):
    '''
    Eliminates dots and substitutes commas for dots in numeric strings
    in order to make them convertible to float
    '''
    y = x.replace(".", "")
    return y.replace(",",".")

def punct_num_str(x):
    '''
    Eliminates dots and substitutes commas for dots in numeric strings
    in order to make them convertible to float
    '''
    try:
        y = str(x).replace(".", "")
        return int(str(y).replace(",","."))
    except:
        y = x.replace(".", "")
    return y.replace(",",".")


def no_dots(x):
    ''' 
    Substitutes strings containing three dots
    for np.nan objects
    '''
    if '...' in str(x):
        return  np.nan
    else:
        pass


# A. Trade flows (Canary)

In [4]:
IC_trade = pd.read_csv ('data/canary_imports_dirty.csv')

In [5]:
IC_trade.head()

,Unnamed: 0,Time,Polonia,Chequia,Suiza,Francia,Italia,Bélgica,Dinamarca,Suecia,Irlanda,Países Bajos,Reino Unido,Alemania
0,0,2021 Noviembre (p),"4.473,862","4.698,110","2.220,403","31.485,825","36.338,226","6.008,680","2.178,621","1.574,086","6.561,863","36.734,327","11.720,918","29.201,971"
1,1,2021 Octubre (p),"3.360,605","9.997,751","1.711,822","7.672,856","12.718,099","4.617,084","2.493,726","1.773,129","3.440,667","29.173,793","6.908,026","25.644,328"
2,2,2021 Septiembre (p),"2.619,986","7.827,948","1.331,296","11.305,727","10.842,908","6.359,599","1.133,544","1.540,708","5.547,320","39.631,925","6.700,103","26.015,663"
3,3,2021 Agosto (p),"2.627,623","3.586,898","1.031,052","7.893,321","13.490,680","11.802,737","1.636,533","1.318,356","5.301,273","16.572,673","4.030,151","39.025,092"
4,4,2021 Julio (p),"2.102,901","10.212,904","1.264,930","7.513,558","16.195,542","10.413,309","1.557,271","1.475,441","4.772,880","18.454,604","3.430,143","35.785,879"


In [6]:
################################################
#OJO - función no importada en uso en esta celda
################################################


IC_trade.Time = IC_trade.Time.apply(del_years)
IC_trade.Time = IC_trade.Time.apply(del_p)
IC_trade.Time = IC_trade.Time.apply(del_years)
IC_trade.Time = IC_trade.Time.apply(del_spaces)
IC_trade.Time = IC_trade.Time.apply(translate_month)
IC_trade.drop(IC_trade.index[IC_trade['Time'] == 'tbd'], inplace=True)
IC_trade.Time = IC_trade.Time.apply(datematcher)
IC_trade.Time = IC_trade.Time.apply(dateconverter)
IC_trade = IC_trade[IC_trade.Time < datetime.date(2020, 1, 7)]
IC_trade = IC_trade.drop('Unnamed: 0', axis=1)

In [7]:
IC_trade.sample(7)

,Time,Polonia,Chequia,Suiza,Francia,Italia,Bélgica,Dinamarca,Suecia,Irlanda,Países Bajos,Reino Unido,Alemania
155,2009-12-07,"1.407,186","1.176,314","2.530,029","10.467,635","14.341,010","6.726,934","2.606,096","13.023,348","4.908,658","31.949,607","13.340,464","49.145,685"
95,2014-07-07,"3.500,730","2.903,990","2.516,900","14.302,180","12.476,447","6.343,674","17.258,817","23.711,638","5.477,799","9.326,740","6.902,781","27.521,471"
74,2016-02-07,"2.403,248","3.174,144","1.967,285","16.547,900","15.402,485","8.120,152","1.982,953","10.940,442","4.950,681","21.496,154","9.711,249","28.931,342"
192,2007-01-07,"2.919,043","1.894,454","1.667,805","14.706,165","16.524,759","7.668,839","2.730,901","4.203,014","6.319,931","14.163,228","15.421,044","43.302,301"
55,2017-08-07,"2.012,037","2.315,483","1.107,910","7.954,201","15.709,094","10.602,966","1.830,528","11.850,965","4.230,436","13.597,325","20.115,973","34.974,042"
188,2007-05-07,"1.807,669","1.680,525","1.736,724","11.936,110","18.765,566","7.750,284","4.694,302","3.524,113","5.603,592","14.842,021","19.128,374","47.321,988"
101,2014-01-07,"2.430,477","2.579,078","2.531,201","8.517,620","11.478,777","50.553,714","2.770,705","20.880,857","2.739,289","32.930,311","9.144,782","26.527,495"


In [8]:
#No hay valores nulos en ninguna columna (guay)
#pd.isnull(serie2).sum()

In [9]:
#Datos de trade flows desde 1/2000 (los datos de turistas de antes del 2000 no nos interesan)

# B. Trade flows (Balearic)

In [10]:
BI_trade = pd.read_csv ('data/balearic_imports_dirty.csv')

In [11]:
BI_trade.head()

,Unnamed: 0,Time,Polonia,República Checa,Francia,Italia,Bélgica,Dinamarca,Suecia,Irlanda,Países Bajos,Reino Unido,Alemania
0,0,2021_Acumulado,7.865.370,1.731.141,123.007.720,230.446.676,53.882.732,4.191.680,8.585.209,4.747.182,178.607.869,42.775.095,196.025.757
1,1,2021M11,1.026.951,66.902,12.992.763,30.885.089,4.247.049,505.164,723.796,1.018.726,35.186.304,3.161.318,23.733.996
2,2,2021M10,719.417,84.056,7.735.753,6.687.123,5.202.484,454.991,1.167.503,331.710,22.533.426,3.540.810,31.231.364
3,3,2021M09,1.183.774,265.947,7.650.884,8.474.734,7.426.750,641.261,1.410.221,1.066.762,25.881.548,3.876.277,24.229.240
4,4,2021M08,548.923,294.308,7.617.475,25.701.516,7.100.870,348.110,927.643,682.895,14.463.688,3.736.112,13.060.456


In [12]:
################################################
#OJO - función no importada en uso en esta celda
################################################


BI_trade.Time = BI_trade.Time.apply(del_total)
BI_trade.Time = BI_trade.Time.apply(del_years)
BI_trade.drop(BI_trade.index[BI_trade['Time'] == 'tbd'], inplace=True)
BI_trade.Time = BI_trade.Time.apply(del_acumulado)
BI_trade.Time = BI_trade.Time.apply(translate_number)
BI_trade.Time = BI_trade.Time.apply(datematcher)
BI_trade.Time = BI_trade.Time.apply(dateconverter)
BI_trade = BI_trade.drop('Unnamed: 0', axis=1)

In [13]:
BI_trade.tail(25)

,Time,Polonia,República Checa,Francia,Italia,Bélgica,Dinamarca,Suecia,Irlanda,Países Bajos,Reino Unido,Alemania
257,2002-02-07,240.248,34.245,99.421.891,11.062.527,908.179,355.756,911.037,4.548,1.033.838,4.763.193,5.102.484
258,2002-01-07,161.812,28.264,5.377.358,3.779.353,1.005.797,308.572,1.229.087,207.895,1.432.924,3.715.025,4.486.436
260,2001-12-07,141.011,56.842,3.670.516,4.216.475,912.521,264.580,688.594,246.547,2.984.561,3.342.475,7.493.955
261,2001-11-07,167.515,39.734,54.588.080,3.952.086,746.866,423.169,1.867.236,172.207,699.818,3.846.682,4.868.650
262,2001-10-07,159.916,45.457,56.142.483,4.297.405,3.788.902,345.754,770.616,231.629,1.402.209,13.554.501,7.223.415
263,2001-09-07,179.688,44.935,4.722.550,4.669.069,1.083.284,577.069,853.360,326.451,955.778,29.903.219,4.662.675
264,2001-08-07,151.048,64.756,5.853.680,6.593.251,1.369.344,694.566,1.229.450,262.383,1.774.065,9.814.252,4.900.638
265,2001-07-07,86.804,61.600,6.154.495,5.109.580,1.475.238,1.075.411,1.798.844,162.357,1.862.962,13.098.362,4.755.641
266,2001-06-07,97.195,56.419,45.177.106,8.762.483,2.630.965,615.401,1.476.777,149.489,1.358.795,14.084.672,28.608.750
267,2001-05-07,117.823,85.638,6.115.289,5.313.264,1.285.194,665.871,1.467.386,273.690,1.573.092,12.637.813,11.474.951


# C. Tourists in the Canaries

In [14]:
#Countries for which there is detailed data for number of visitors and imports
countries = ['UK', 'HOLANDA', 'IRLANDA', 'SUECIA', 'BELGICA', 'ITALIA', 'FRANCIA', 'REPCHECA', 'POLONIA', 'ALEMANIA', 'SUIZA', 'DINAMARCA']
countries_noger = ['UK', 'HOLANDA', 'IRLANDA', 'SUECIA', 'BELGICA', 'ITALIA', 'FRANCIA', 'REPCHECA', 'POLONIA', 'SUIZA', 'DINAMARCA']

In [15]:
#Definir una función para apendear en una columna nueva a cada df de los definidos 
#abajo el número de turistas desde los dfs de arriba
#puede que para ello tengas que dropear la columna total

Polonia_IC = pd.DataFrame(IC_trade[['Time','Polonia']])
Polonia_IC = Polonia_IC.rename({'Polonia': 'Imports'}, axis=1)
Polonia_IC['Country'] = ['PLN' for v in range (0,240)]

República_Checa_IC = pd.DataFrame(IC_trade[['Time','Chequia']])
República_Checa_IC = República_Checa_IC.rename({'Chequia': 'Imports'}, axis=1)
República_Checa_IC['Country'] = ['CZE' for v in range (0,240)]

Francia_IC = pd.DataFrame(IC_trade[['Time','Francia']])
Francia_IC = Francia_IC.rename({'Francia': 'Imports'}, axis=1)
Francia_IC['Country'] = ['FRA' for v in range (0,240)]

Italia_IC = pd.DataFrame(IC_trade[['Time','Italia']])  
Italia_IC = Italia_IC.rename({'Italia': 'Imports'}, axis=1)
Italia_IC['Country'] = ['ITA' for v in range (0,240)]

Bélgica_IC = pd.DataFrame(IC_trade[['Time','Bélgica']])  
Bélgica_IC = Bélgica_IC.rename({'Bélgica': 'Imports'}, axis=1)
Bélgica_IC['Country'] = ['BEL' for v in range (0,240)]

Suecia_IC = pd.DataFrame(IC_trade[['Time','Suecia']])
Suecia_IC = Suecia_IC.rename({'Suecia': 'Imports'}, axis=1)
Suecia_IC['Country'] = ['SWE' for v in range (0,240)]

Irlanda_IC = pd.DataFrame(IC_trade[['Time','Irlanda']])  
Irlanda_IC = Irlanda_IC.rename({'Irlanda': 'Imports'}, axis=1)
Irlanda_IC['Country'] = ['IRL' for v in range (0,240)]

Países_Bajos_IC = pd.DataFrame(IC_trade[['Time','Países Bajos']]) 
Países_Bajos_IC = Países_Bajos_IC.rename({'Países Bajos': 'Imports'}, axis=1)
Países_Bajos_IC['Country'] = ['NLD' for v in range (0,240)]

Reino_Unido_IC = pd.DataFrame(IC_trade[['Time','Reino Unido']]) 
Reino_Unido_IC = Reino_Unido_IC.rename({'Reino Unido': 'Imports'}, axis=1)
Reino_Unido_IC['Country'] = ['GBR' for v in range (0,240)]

Alemania_IC = pd.DataFrame(IC_trade[['Time','Alemania']])
Alemania_IC = Alemania_IC.rename({'Alemania': 'Imports'}, axis=1)
Alemania_IC['Country'] = ['GER' for v in range (0,240)]

Dinamarca_IC = pd.DataFrame(IC_trade[['Time','Dinamarca']])
Dinamarca_IC = Dinamarca_IC.rename({'Dinamarca': 'Imports'}, axis=1)
Dinamarca_IC['Country'] = ['DEN' for v in range (0,240)]

Suiza_IC = pd.DataFrame(IC_trade[['Time','Suiza']])
Suiza_IC = Suiza_IC.rename({'Suiza': 'Imports'}, axis=1)
Suiza_IC['Country'] = ['SWI' for v in range (0,240)]


with ExcelWriter('data/support_tourist.xlsx') as writer: 
    
    Polonia_IC.to_excel(writer, sheet_name='POLONIA')
    República_Checa_IC.to_excel(writer, sheet_name='REPCHECA')
    Francia_IC.to_excel(writer, sheet_name='FRANCIA')
    Italia_IC.to_excel(writer, sheet_name='ITALIA') 
    Bélgica_IC.to_excel(writer, sheet_name='BELGICA') 
    Suecia_IC.to_excel(writer, sheet_name='SUECIA')
    Irlanda_IC.to_excel(writer, sheet_name='IRLANDA')
    Países_Bajos_IC.to_excel(writer, sheet_name='HOLANDA')
    Reino_Unido_IC.to_excel(writer, sheet_name='UK')
    Alemania_IC.to_excel(writer, sheet_name='ALEMANIA') 
    
    Suiza_IC.to_excel(writer, sheet_name='SUIZA')
    Dinamarca_IC.to_excel(writer, sheet_name='DINAMARCA')
                        

  
                              

In [16]:
#MACROLOOP FUNCIONAL (COMENTAR EN INGLÉS...)

for c in countries:
    df_A = pd.read_excel('data/TuristasCI_Short.xls', sheet_name= f'{c}')
    df_B = pd.read_excel('data/support_tourist.xlsx', sheet_name= f'{c}')
    df_A.drop(labels='TOTAL', inplace=True, axis=1)
    df_A.rename(columns={"Unnamed: 0":"Year"}, inplace=True)
    df_A = df_A[df_A.Year > 1999]
    df_A = df_A[df_A.Year < 2020]
    
    df_AT = df_A.T   

    df_AT.columns = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']
    df_AT = df_AT.reset_index()
    df_AT.drop(labels='index', inplace=True, axis=1)
    df_AT = df_AT.iloc[1: , :]
    
    df_C = df_AT['1']
    
    for i in range(2,21):
        df_D = df_AT[f'{i}']
        df_C = pd.concat([df_C, df_D])
        
    df_C = df_C.to_frame()
    df_C.columns = ['Column_A']
    print(f'Saving {c}...')
    df_C = df_C.reset_index()
    df_B['Tourist'] = df_C['Column_A']
    df_B = df_B.reset_index()
    df_B.rename(columns={"Unnamed: 0":"Delete"}, inplace=True)
    df_B.drop(labels='Delete', inplace=True, axis=1)
    lista_B = df_B['Tourist'].to_list()
    lista_B.reverse()
    df_B['Tourists'] = lista_B
    df_B.drop(labels='Tourist', inplace=True, axis=1)

    with ExcelWriter(f'data/support_tourist_folder/{c}.xlsx') as writer:
        df_B.to_excel(writer, sheet_name= f'{c}')

Saving UK...
Saving HOLANDA...
Saving IRLANDA...
Saving SUECIA...
Saving BELGICA...
Saving ITALIA...
Saving FRANCIA...
Saving REPCHECA...
Saving POLONIA...
Saving ALEMANIA...
Saving SUIZA...
Saving DINAMARCA...


In [17]:
df_X = pd.read_excel('data/support_tourist_folder/ALEMANIA.xlsx')
for c in countries_noger:
    df_Y = pd.read_excel(f'data/support_tourist_folder/{c}.xlsx')
    df_X = pd.concat([df_X, df_Y])

df_X.rename(columns={"Unnamed: 0":"Delete"}, inplace=True)
df_X.drop(labels='Delete', inplace=True, axis=1)
df_X.rename(columns={"index":"Delete"}, inplace=True)
df_X.drop(labels='Delete', inplace=True, axis=1)
df_X['Year'] = df_X['Time'].dt.year
df_X['Imports'] = df_X['Imports'].apply(punct_num)
df_X["Imports"] = pd.to_numeric(df_X["Imports"], downcast="float")

display(df_X)

,Time,Imports,Country,Tourists,Year
0,2019-12-07,36917.476562,GER,270865,2019
1,2019-11-07,35035.882812,GER,270307,2019
2,2019-10-07,50706.667969,GER,213744,2019
3,2019-09-07,23587.550781,GER,182978,2019
4,2019-08-07,30583.541016,GER,187302,2019
...,...,...,...,...,...
235,2000-05-07,3160.726074,DEN,8718,2000
236,2000-04-07,2636.031982,DEN,21934,2000
237,2000-03-07,3576.811035,DEN,32809,2000
238,2000-02-07,2452.499023,DEN,31644,2000


In [18]:
#dataframe = pd.DataFrame(df_X["Tourists","Year"].groupby(df_X["Country", ]).sum())
df_X_yearly = df_X.groupby(["Year", "Country"]).sum()

display(df_X_yearly)


Imports  Tourists
Year Country                         
2000 BEL       73257.781250    230560
     CZE       21414.966797     69661
     DEN       41528.562500    251519
     FRA      169747.531250    164103
     GBR      225495.390625   3713980
...                     ...       ...
2019 ITA      236307.859375    476554
     NLD      423757.750000    594075
     PLN       43338.718750    271576
     SWE       91024.359375    483349
     SWI       22513.324219    313083

[240 rows x 2 columns]

# D. Tourists in the Balearics

In [19]:
countries_nosui = ['Reino Unido', 'Paises Bajos', 'Belgica', 'Italia', 'Francia', 'Alemania']
countries_noger_nosui = ['Reino Unido', 'Paises Bajos', 'Belgica', 'Italia', 'Francia']
country_sample = ['Belgica','Francia' 'Italia', 'Paises Bajos', 'Reino Unido', 'Alemania', 'Suiza' ,'Rusia']

In [20]:
#adaptado del código de canarias usando métodos de strings (mencionar en inglés)

Polonia_BI = pd.DataFrame(BI_trade[['Time','Polonia']])
Polonia_BI = Polonia_BI.rename({'Polonia': 'Imports'}, axis=1)
Polonia_BI['Country'] = ['PLN' for v in range (0,263)]

República_Checa_BI = pd.DataFrame(BI_trade[['Time','República Checa']])
República_Checa_BI = República_Checa_BI.rename({'Chequia': 'Imports'}, axis=1)
República_Checa_BI['Country'] = ['CZE' for v in range (0,263)]

Francia_BI = pd.DataFrame(BI_trade[['Time','Francia']])
Francia_BI = Francia_BI.rename({'Francia': 'Imports'}, axis=1)
Francia_BI['Country'] = ['FRA' for v in range (0,263)]

Italia_BI = pd.DataFrame(BI_trade[['Time','Italia']])  
Italia_BI = Italia_BI.rename({'Italia': 'Imports'}, axis=1)
Italia_BI['Country'] = ['ITA' for v in range (0,263)]

Bélgica_BI = pd.DataFrame(BI_trade[['Time','Bélgica']])  
Bélgica_BI = Bélgica_BI.rename({'Bélgica': 'Imports'}, axis=1)
Bélgica_BI['Country'] = ['BEL' for v in range (0,263)]

Suecia_BI = pd.DataFrame(BI_trade[['Time','Suecia']])
Suecia_BI = Suecia_BI.rename({'Suecia': 'Imports'}, axis=1)
Suecia_BI['Country'] = ['SWE' for v in range (0,263)]

Irlanda_BI = pd.DataFrame(BI_trade[['Time','Irlanda']])  
Irlanda_BI = Irlanda_BI.rename({'Irlanda': 'Imports'}, axis=1)
Irlanda_BI['Country'] = ['IRL' for v in range (0,263)]

Países_Bajos_BI = pd.DataFrame(BI_trade[['Time','Países Bajos']]) 
Países_Bajos_BI = Países_Bajos_BI.rename({'Países Bajos': 'Imports'}, axis=1)
Países_Bajos_BI['Country'] = ['NLD' for v in range (0,263)]

Reino_Unido_BI = pd.DataFrame(BI_trade[['Time','Reino Unido']]) 
Reino_Unido_BI = Reino_Unido_BI.rename({'Reino Unido': 'Imports'}, axis=1)
Reino_Unido_BI['Country'] = ['GBR' for v in range (0,263)]

Alemania_BI = pd.DataFrame(BI_trade[['Time','Alemania']])
Alemania_BI = Alemania_BI.rename({'Alemania': 'Imports'}, axis=1)
Alemania_BI['Country'] = ['GER' for v in range (0,263)]

Dinamarca_BI = pd.DataFrame(BI_trade[['Time','Dinamarca']])
Dinamarca_BI = Dinamarca_BI.rename({'Dinamarca': 'Imports'}, axis=1)
Dinamarca_BI['Country'] = ['DEN' for v in range (0,263)]



with ExcelWriter('data/support_tourist_2.xlsx') as writer: 
   
    Polonia_BI.to_excel(writer, sheet_name='POLONIA')
    República_Checa_BI.to_excel(writer, sheet_name='REPCHECA')
    Francia_BI.to_excel(writer, sheet_name='Francia')
    Italia_BI.to_excel(writer, sheet_name='Italia') 
    Bélgica_BI.to_excel(writer, sheet_name='Belgica') 
    Suecia_BI.to_excel(writer, sheet_name='SUECIA')
    Irlanda_BI.to_excel(writer, sheet_name='IRLANDA')
    Países_Bajos_BI.to_excel(writer, sheet_name='Paises Bajos')
    Reino_Unido_BI.to_excel(writer, sheet_name='Reino Unido')
    Alemania_BI.to_excel(writer, sheet_name='Alemania') 
   
    Dinamarca_BI.to_excel(writer, sheet_name='DINAMARCA')

In [21]:
df_K = pd.read_csv('data/TuristasBI_Short_C.csv', sep=';')


df_K.rename(columns={"Unnamed: 0":"Time"}, inplace=True)
df_K["Time"] = df_K["Time"].apply(translate_number)
df_K["Time"] = df_K["Time"].apply(datematcher)
df_K = df_K[df_K.Time > datetime.datetime(2005,1,1)]
df_K = df_K[df_K.Time < datetime.datetime(2020,1,1)]

df_K = df_K.replace('...', np.nan)
df_K.drop(labels='Rusia', inplace=True, axis=1)
df_K = df_K.reset_index()
df_K.rename(columns={"index":"Delete"}, inplace=True)
df_K.drop(labels='Delete', inplace=True, axis=1)
 

display(df_K)

df_K.isna().sum()


,Time,Belgica,Francia,Italia,Paises Bajos,Reino Unido,Alemania,Suiza
0,2019-12-07,1846,8395,3419,2933,15391,69008,3878
1,2019-11-07,1975,8608,5582,5557,15415,108237,5966
2,2019-10-07,14214,64842,29927,48883,337509,466520,45825
3,2019-09-07,35823,94693,102521,76201,606611,645933,62384
4,2019-08-07,41162,161094,192856,108556,646813,593696,61803
...,...,...,...,...,...,...,...,...
175,2005-05-07,15555,38240,35180,28198,454937,454274,23962
176,2005-04-07,8110,40556,7547,3532,145653,231734,12940
177,2005-03-07,3605,9634,3869,2133,87277,233349,6693
178,2005-02-07,1722,5258,2107,1000,44718,160791,5363


Time            0
Belgica         0
Francia         0
Italia          0
Paises Bajos    0
Reino Unido     0
Alemania        0
Suiza           0
dtype: int64

In [22]:
#MACROLOOP FUNCIONAL (COMENTAR EN INGLÉS...)

for c in countries_nosui:
    #df_L = pd.read_excel('data/TuristasBI_Short_C.xls', )
    df_L = pd.read_excel('data/support_tourist_2.xlsx', sheet_name= f'{c}')
    df_L = df_L[df_L.Time > datetime.datetime(2005,1,1)]
    df_L = df_L[df_L.Time < datetime.datetime(2020,1,1)]
    df_L = df_L.reset_index()
    df_L.rename(columns={"Unnamed: 0":"Delete"}, inplace=True)
    df_L.drop(labels='Delete', inplace=True, axis=1)
    df_L.rename(columns={"index":"Delete"}, inplace=True)
    df_L.drop(labels='Delete', inplace=True, axis=1)
    df_L['Tourist'] = df_K[f'{c}']
    df_L['Imports'] = df_L['Imports'].apply(punct_num)
    

    with ExcelWriter(f'data/support_tourist_folder_2/{c}.xlsx') as writer:
        df_L.to_excel(writer, sheet_name= f'{c}')

/home/diego/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:211: UserWarning: Cell B2 is marked as a date but the serial value -693594 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/home/diego/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:211: UserWarning: Cell B2 is marked as a date but the serial value -693594 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/home/diego/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:211: UserWarning: Cell B2 is marked as a date but the serial value -693594 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/home/diego/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:211: UserWarning: Cell B2 is marked as a date but the serial value -693594 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)
/home/diego/anaconda3/lib/python3.9/site-package

In [23]:
df_M = pd.read_excel('data/support_tourist_folder_2/Alemania.xlsx')
for c in countries_noger_nosui:
    df_N = pd.read_excel(f'data/support_tourist_folder_2/{c}.xlsx')
    df_M = pd.concat([df_M, df_N])

df_M['Year'] = df_M['Time'].dt.year


display(df_M)

,Unnamed: 0,Time,Imports,Country,Tourist,Year
0,0,2019-12-07,24961994,GER,69008,2019
1,1,2019-11-07,20791329,GER,108237,2019
2,2,2019-10-07,21244446,GER,466520,2019
3,3,2019-09-07,22357741,GER,645933,2019
4,4,2019-08-07,20255555,GER,593696,2019
...,...,...,...,...,...,...
175,175,2005-05-07,34803435,FRA,38240,2005
176,176,2005-04-07,4768458,FRA,40556,2005
177,177,2005-03-07,7480924,FRA,9634,2005
178,178,2005-02-07,4274270,FRA,5258,2005


# C. Foreign citizens in the Canaries and the Balearic islands